<a href="https://colab.research.google.com/github/estebanhramirez/Google-Colab/blob/main/Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [487]:
import pandas as pd
import xlrd
import numpy as np
from scipy.optimize import linprog

**Interface**

In [488]:

def excel2matrix(doc_name, sheet_name):
  df = pd.read_excel(doc_name, sheet_name, index_col=0)
  arr = df.to_numpy()
  result = [inner for outer in arr for inner in outer]
  return (result)


**Middleware**

In [489]:

def pretty_print_weekly(x, items):
  M = []

  ctr = 0
  row = []
  for i in range(1, len(x)+1):
    row.append(x[i-1])
    if (i % 7 == 0):
      row.insert(0, items[ctr])
      M.append(row)
      ctr += 1
      row = []

  day = [[' ', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']]
  print('\n'.join([''.join(['{:8}'.format(ele) for ele in row]) for row in day]))
  print('\n'.join([''.join(['{:8}'.format(ele) for ele in row]) for row in M]))


In [490]:

def build_c(prices):
  c = []
  for p in prices:
    for i in range(0, 7):
      c += [p]
  return c


In [491]:

def constrain_by_interface(A, b, sign, constrains):
  n = int(len(constrains)/7)
  for i, entry in enumerate(constrains):
    a = [0]*(7*n)
    a[i] = sign
    A.append(a)
    b += [sign*entry]
  return (A, b)

def constrain_weekly_min(A, b, idx_item, min_amount, n_items):
  A.append([-1 for i in range(0, 7*n_items)])
  b += [-min_amount]
  return (A, b)

def constrain_weekly_max(A, b, idx_item, max_amount, n_items):
  A.append([1 for i in range(0, 7*n_items)])
  b += [max_amount]
  return (A, b)

def constrain_weekly_max_item(A, b, idx_item, max_amount, n_items):
  a = [0]*(7*n_items)
  for i in range(0, 7):
    a[(7*idx_item)+i] = 1
  A.append(a)
  b += [max_amount]
  return (A, b)

def constrain_daily_max_item(A, b, idx_item, idx_day, max_amount, n_items):
  a = [0]*(7*n_items)
  a[(7*idx_item)+idx_day] = 1
  A.append(a)
  b += [max_amount]
  return (A, b)


In [492]:

def find_min_cost_local(c, A_ub, b_ub, A_eq, b_eq, bounds, items, verbose=False):
  res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
  if res.fun is not None and verbose:
    print(res.message)
    print('raw printing..')
    print(res.x)
    print('\n'+"pretty printing..")
    pretty_print_weekly(res.x, items)
  return (res.fun)

def find_min_cost_global(A_ub, b_ub, A_eq, b_eq, items, prices_markets, prices, path, it):
  if it == len(items):
    c = build_c(prices)
    A_ub = [[0]*(7*len(prices))]
    A_eq = [[0]*(7*len(prices))]
    b_ub = [0]
    b_eq = [0]
    bounds = [(1, None)]*(7*len(prices))

    opt = find_min_cost_local(c, A_ub, b_ub, A_eq, b_eq, bounds=bounds, items=items)
    return (opt, prices, path)
  else:
    global_min = np.inf
    prices_min = []
    path_min = []
    for i, prices_market in enumerate(prices_markets):
      local_min, prices_min_, path_min_ = find_min_cost_global(A_ub, b_ub, A_eq, b_eq, items, prices_markets, prices+[prices_market[it]], path+[i], it+1)
      if local_min is None:
        local_min = np.inf
      if local_min < global_min:
        global_min = local_min
        prices_min = prices_min_
        path_min = path_min_
    return (global_min, prices_min, path_min)


**Beggining of execution**

In [493]:

items_run = ['Appel', 'Garlic', 'Onion', 'Potato', 'Rice', 'Tomato']

prices_market1 = [
            2000, # Appel
            1800, # Garlic
            6000, # Onion
            3000, # Potato
            3500, # Rice
            1500, # Tomato
          ]

prices_market2 = [
            18, # Appel
            18, # Garlic
            62, # Onion
            32, # Potato
            34, # Rice
            1900, # Tomato
          ]

prices_market3 = [
            100, # Appel
            1500, # Garlic
            600, # Onion
            4000, # Potato
            60, # Rice
            900, # Tomato
          ]


**Test suite**

In [494]:
n_run = len(prices_market1)

c_run = build_c(prices_market2)

A_ub_run = [[0]*(7*n_run)]
A_eq_run = [[0]*(7*n_run)]
b_ub_run = [0]
b_eq_run = [0]

# Restrictions only <=
A_ub_run, b_ub_run = constrain_by_interface(A_ub_run, b_ub_run, -1, excel2matrix("Planning User Interface.xlsx", "Lower limits"))
A_ub_run, b_ub_run = constrain_by_interface(A_ub_run, b_ub_run, 1, excel2matrix("Planning User Interface.xlsx", "Upper limits"))

# Restrictions only ==

A_eq_run, b_eq_run = constrain_weekly_max_item(A_eq_run, b_eq_run, 0, 100, n_run)
A_eq_run, b_eq_run = constrain_weekly_max_item(A_eq_run, b_eq_run, 1, 105, n_run)
A_eq_run, b_eq_run = constrain_weekly_max_item(A_eq_run, b_eq_run, 2, 120, n_run)
A_eq_run, b_eq_run = constrain_weekly_max_item(A_eq_run, b_eq_run, 3, 125, n_run)
A_eq_run, b_eq_run = constrain_weekly_max_item(A_eq_run, b_eq_run, 4, 99, n_run)

#A_eq_run, b_eq_run = constrain_daily_max_item(A_eq_run, b_eq_run, 3, 0, 10, n_run)
#A_eq_run, b_eq_run = constrain_daily_max_item(A_eq_run, b_eq_run, 3, 1, 3, n_run)
#A_eq_run, b_eq_run = constrain_daily_max_item(A_eq_run, b_eq_run, 5, 2, 100, n_run)
#A_eq_run, b_eq_run = constrain_daily_max_item(A_eq_run, b_eq_run, 2, 5, 222, n_run)

print(c_run)
print(A_eq_run, b_eq_run)


bounds_run = [(0, None)]*(7*n_run)

[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 62, 62, 62, 62, 62, 62, 62, 32, 32, 32, 32, 32, 32, 32, 34, 34, 34, 34, 34, 34, 34, 1900, 1900, 1900, 1900, 1900, 1900, 1900]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]] [0, 100, 105, 120, 125, 99]


In [495]:
price = find_min_cost_local(c_run, A_ub=A_ub_run, b_ub=b_ub_run, A_eq=A_eq_run, b_eq=b_eq_run, bounds=bounds_run, items=items_run, verbose=True)
print('Optimal price:', price)

Optimization terminated successfully. (HiGHS Status 7: Optimal)
raw printing..
[40. 10. 10. 10. 10. 10. 10. 10. 10. 10. 15. 20. 20. 20. 10. 10. 10. 10.
 30. 20. 30. 65. 10. 10. 10. 10. 10. 10. 39. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10.]

pretty printing..
        Mon     Tue     Wed     Thu     Fri     Sat     Sun     
Appel       40.0    10.0    10.0    10.0    10.0    10.0    10.0
Garlic      10.0    10.0    10.0    15.0    20.0    20.0    20.0
Onion       10.0    10.0    10.0    10.0    30.0    20.0    30.0
Potato      65.0    10.0    10.0    10.0    10.0    10.0    10.0
Rice        39.0    10.0    10.0    10.0    10.0    10.0    10.0
Tomato      10.0    10.0    10.0    10.0    10.0    10.0    10.0
Optimal price: 151496.0


In [496]:
find_min_cost_global(A_ub_run, b_ub_run, A_eq_run, b_eq_run, items_run, [prices_market1, prices_market2, prices_market3], [], [], 0)

(7448.0, [18, 18, 62, 32, 34, 900], [1, 1, 1, 1, 1, 2])